In [4]:
import json
import importlib  
import os
import matplotlib.pyplot as plt
os.chdir(os.path.join('.', '..'))
import utils
import analysis_utils
import pandas as pd
import numpy as np
import re
analyze_data = importlib.import_module("analyze-data")

usr = os.path.expanduser('~')

def experiment_results_frame(input_file):
    output_file = os.path.join(experiment_path(experiment), "inference_turn_info.csv")
    baseline_test_set = analysis_utils.read_json(input_file)
    frame = analyze_data.generate_TRADE_turn_frame(baseline_test_set)
    return frame

def experiment_path(experiment):
    return os.path.join(usr,''.join(("misc_trade/results/", experiment)))

dom_list = ["hotel", "taxi", "train", "restaurant", "attraction"]

split = "dev" 
df = analyze_data.dials_as_frame(split, dom_list)

experiment = "baseline21/full"
input_file = os.path.join(experiment_path(experiment), ''.join(("prediction_TRADE_".split,".json")))    
df_pred = experiment_results_frame(input_file)

In [5]:
experiment = "baseline21/full"
input_file = os.path.join(experiment_path(experiment), ''.join(("prediction_TRADE_".split,".json")))    
df_pred = experiment_results_frame(input_file)

In [ ]:

plot_error_histograms(df_pred, experiment)
plot_empty_beliefs(df_pred, experiment)

In [ ]:
import numpy as np
import json
import os
import re
import pandas as pd
from analysis_utils import *
import matplotlib.pyplot as plt
usr = os.path.expanduser('~')
import importlib  
analyze_data = importlib.import_module("analyze-data")
    
max_turn_len = 17
all_slots = read_csv(os.path.join(usr,"data/trade/all_slots.txt"))

experiments = ["baseline", "except-attraction", "except-hotel", "except-taxi", "except-train", 
              "only-attraction", "only-hotel", "only-taxi", "only-train"]

def experiment_results_frame(input_file):
    output_file = os.path.join(experiment_path(experiment), "inference_turn_info.csv")
    baseline_test_set = read_json(input_file)
    frame = generate_turn_frame(baseline_test_set)
    return frame

In [ ]:
for experiment in ["baseline21/full"]:
    input_file = os.path.join(experiment_path(experiment), "prediction_TRADE_dev.json")    
    df = experiment_results_frame(input_file)
    plot_error_histograms(df, experiment)
    plot_empty_beliefs(df, experiment)
#df.to_csv(os.path.join(usr, output_file))

In [ ]:
#df.head()
#df.pred_full_belief

## Plot the number of examples with a particular length

In [ ]:
k = df[["dialogue", "turn"]]
k.turn = k.turn.astype(int)
dialogue_lengths = k.groupby("dialogue").max().values.astype(int)
sample_lengths = df["turn"].values.astype(int)

max_turn_len = max(df["turn"].values.astype(int))
plt.hist(sample_lengths, alpha=0.5, label='sample lengths', histtype='stepfilled')
plt.hist(dialogue_lengths, alpha=0.5, label='dialogue lengths', histtype='stepfilled')
plt.xticks(list(range(max_turn_len)))
plt.legend(loc='upper right')
plt.title('Histogram of the number of turns in samples, dialogues')
l = plt.plot()

## Plot the percentage of samples correct at turn-level
If the correct labels are [A, B, C, D, E] and predicted labels are [A, B, C, F, G, H], then fraction of correct samples is 0.6 ([True, True, True, False, False])

In [ ]:
left = df[["turn", "full_correct"]].astype(int).groupby("turn").sum()
left["total"] = df[["turn", "full_correct"]].astype(int).groupby("turn").count().values

In [ ]:
left

In [ ]:
plt.plot(df[["turn", "full_correct"]].astype(int).groupby("turn").mean())
plt.show()

In [ ]:
a = df[["turn", "full_correct"]].astype(int).groupby("turn").mean()
a["n_examples"] = df[["turn", "full_correct"]].astype(int).groupby("turn").size()

In [ ]:
a["error"] = a["full_correct"].apply(lambda x: 1-x)

In [ ]:
a["error_weighed"] = a["error"]*a["n_examples"]

In [ ]:
a["turn"] = a.index

In [ ]:
plt.plot(a[["turn", "error_weighed"]].astype(int).groupby("turn").mean())
plt.show()

In [ ]:
a["error"].values

## Accuracy vs number of slots i the last turn

In [ ]:
df[["turn", "full_correct", "true_full_belief"]].head()

In [ ]:
df["n_slots_in_step"] = df["true_step_belief"].apply(len)
df["n_accumulative_slots"] = df["true_full_belief"].apply(len)-df["n_slots_in_step"]

In [ ]:
a = df[["n_slots_in_step", "n_accumulative_slots", "full_correct"]].astype(int)
print(a.head())
error = a.astype(int).groupby(["n_accumulative_slots","n_slots_in_step"]).mean().apply(lambda x: 1-x)
freq = a.astype(int).groupby(["n_accumulative_slots", "n_slots_in_step"]).size()
weighted_errors = error.values.reshape(-1)*freq.values.reshape(-1)

In [ ]:
error["freq"] = freq
error["error"] = error["full_correct"]
del error["full_correct"]
error["num_errors"] = weighted_errors

In [ ]:
error

In [ ]:
error_number = error.groupby("n_accumulative_slots").sum()["num_errors"].sum()
plt.plot(error.groupby("n_accumulative_slots").sum()["num_errors"]/error_number)

In [ ]:
acc_vs_num_past_slots = df[["n_slots_in_step", "full_correct"]].astype(int).groupby("n_slots_in_step").mean()
plt.plot(acc_vs_num_past_slots)
plt.show()
acc_vs_num_past_slots["n_examples"] = list(df[["n_slots_in_step", "full_correct"]].astype(int).groupby("n_slots_in_step").size())

In [ ]:
# acc_vs_num_past_slots = df[["n_slots_in_step", "full_correct"]].astype(int).groupby("n_slots_in_step").mean()
# acc_vs_num_past_slots["n_examples"] = df[["n_slots_in_step", "full_correct"]].astype(int).groupby("n_slots_in_step").size()
# plt.plot(df[["n_slots_in_step", "full_correct"]])
# plt.show()

In [ ]:
acc_vs_num_past_slots = df[["n_slots_in_step", "full_correct"]].astype(int).groupby("n_slots_in_step").mean()
acc_vs_num_past_slots["n_examples"] = df[["n_slots_in_step", "full_correct"]].astype(int).groupby("n_slots_in_step").size()
plt.plot(df[["n_slots_in_step", "full_correct"]])

In [ ]:
acc_vs_num_past_slots

In [ ]:
acc_vs_num_past_slots = df[["n_slots_in_step", "full_correct"]].astype(int).groupby("n_slots_in_step").mean()
plt.plot(acc_vs_num_past_slots)
plt.show()
acc_vs_num_past_slots["n_examples"] = list(df[["n_slots_in_step", "full_correct"]].astype(int).groupby("n_slots_in_step").size())

In [ ]:
acc_vs_num_past_slots

In [ ]:
error_change = acc_vs_num_past_slots.values

In [ ]:
acc_vs_num_past_slots.values.reshape(16)

In [ ]:
set(true)

## Joint accuracy vs turn

In [ ]:
df[["turn", "full_correct"]].head()

## Percent slots correct of those present

In [ ]:
def copy_under_condition(df, condition, name):
    df.loc[:,name] = condition
    df[name] = df[name].mask(condition == True, df.turn).astype(int)
    df[name] = df[name].mask(condition == False, None)
    return df

In [ ]:

df_correct = df["det_full_correct"].apply(sum).astype(float)
df_slots = df["det_full_correct"].apply(len).astype(float)
df["percent_correct"] = df_correct/df_slots
a = df[["turn", "percent_correct"]]
partially_correct = a[(a["percent_correct"]>0) & (a["percent_correct"] < 1)]
fully_correct = a[a["percent_correct"] == 1]
fully_incorrect = a[a["percent_correct"] == 0]
correct_empty = a[a["percent_correct"].isna()]

# a = copy_under_condition(a, (a["percent_correct"]>0) & (a["percent_correct"] < 1), name="partially_correct")
# a = copy_under_condition(a, a["percent_correct"] == 1, name="fully_correct")
# a = copy_under_condition(a, a["percent_correct"] == 0, name="fully_incorrect")
# a = copy_under_condition(a, a["percent_correct"].isna(), name="correct_empty")

# a["partially_correct"] = (a["percent_correct"]>0) & (a["percent_correct"] < 1)
# a["fully_correct"] = a["percent_correct"] == 1
# a["fully_incorrect"] = a["percent_correct"] == 0
# a["correct_empty"] = a["percent_correct"].isna()

In [ ]:
plot_error_histograms(df)

In [ ]:
import numpy as np

plt.hist(fully_correct.turn.values.astype(int), alpha=0.2, label='correct', histtype='stepfilled')
plt.hist(partially_correct.turn.values.astype(int), alpha=0.3, label='partial', histtype='stepfilled')
plt.hist(correct_empty.turn.values.astype(int), alpha=0.7, label='correct empty', histtype='bar', rwidth=2)
plt.hist(fully_incorrect.turn.values.astype(int), bins = 4, alpha=0.5, label='fully incorrect', histtype='stepfilled')
plt.xticks(list(range(max_turn_len)))
#plt.yscale('log')
plt.legend(loc='upper right')
plt.show()

In [ ]:
percent_correct = df["percent_correct"]
n_slots = percent_correct.shape[0]
n_empty = (percent_correct).isna().sum()
n_fully_correct = percent_correct[percent_correct==1].sum()
n_partially_correct = percent_correct[(percent_correct!=1)&(percent_correct!=0)].shape[0]
#n_completely_wrong = n_slots-n_correct-n_partially_correct

In [ ]:
n_empty, n_fully_correct, n_partially_correct, n_slots

In [ ]:
n_fully_correct/n_slots # this seems incorrect...

In [ ]:
percent_correct = n_correct/n_slots